In [1]:
!pip install pyspark findspark

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [4]:
spark = (
    SparkSession.builder.master('local[*]')
    .appName('Aula Pivot e Unpivot')
    .config('spark.sql.legacy.timeParserPolicy', 'LEGACY')
    .getOrCreate()
)

24/05/03 09:02:44 WARN Utils: Your hostname, casa-Default-string resolves to a loopback address: 127.0.1.1; using 192.168.0.9 instead (on interface enp7s0)
24/05/03 09:02:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/05/03 09:02:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

In [ ]:
compras = spark.read.parquet('./DATASETS/COMPRAS.parquet')

In [33]:
df2 = (
    compras
    .withColumn('mes', F.date_format('data', 'MMMM'))
    .groupBy('cartao_bandeira')
    .pivot('mes', ['January', 'February'])
    .agg(F.count('*').alias('qnt'))
)

In [34]:
df2.show()

+--------------------+-------+--------+
|     cartao_bandeira|January|February|
+--------------------+-------+--------+
|       VISA 16 digit|    878|     772|
|       VISA 13 digit|    420|     350|
|            Discover|    454|     406|
|Diners Club / Car...|    397|     395|
|    American Express|    385|     353|
|             Maestro|    424|     374|
|          Mastercard|    415|     359|
|        JCB 16 digit|    835|     756|
|       VISA 19 digit|    437|     388|
|        JCB 15 digit|    386|     374|
+--------------------+-------+--------+



In [39]:
# unpivot
df2.select(F.col('cartao_bandeira') ,F.expr('stack(2, "Jan", January, "Feb", February) as (mes, valor)')).show()

+--------------------+---+-----+
|     cartao_bandeira|mes|valor|
+--------------------+---+-----+
|       VISA 16 digit|Jan|  878|
|       VISA 16 digit|Feb|  772|
|       VISA 13 digit|Jan|  420|
|       VISA 13 digit|Feb|  350|
|            Discover|Jan|  454|
|            Discover|Feb|  406|
|Diners Club / Car...|Jan|  397|
|Diners Club / Car...|Feb|  395|
|    American Express|Jan|  385|
|    American Express|Feb|  353|
|             Maestro|Jan|  424|
|             Maestro|Feb|  374|
|          Mastercard|Jan|  415|
|          Mastercard|Feb|  359|
|        JCB 16 digit|Jan|  835|
|        JCB 16 digit|Feb|  756|
|       VISA 19 digit|Jan|  437|
|       VISA 19 digit|Feb|  388|
|        JCB 15 digit|Jan|  386|
|        JCB 15 digit|Feb|  374|
+--------------------+---+-----+

